# Gastebois Jacques 
## Benchmarking : Validation de la Distillation sur Nouvelles Architectures

Ce notebook a pour but de valider que nos images distilles ne sont pas "sur-apprises" pour un seul modele, mais qu'elles peuvent servir de base d'entrainement pour n'importe quel reseau.

Le but etant de fournir des outils robustes et verifies.

---

### 1. chargement des outils d'evaluation

on reprend les utilitaires du papier original pour etre sur de pas dire de betises sur les chiffres. j'ai un peu fait le menage dans les imports mais ca reste un peu le bazar. :)

In [ ]:
!pip install -q huggingface_hub

import torch
import torch.nn as  nn
import numpy as  np
import os 
from huggingface_hub import hf_hub_download
from utils import get_dataset, get_network, evaluate_synset

if torch.cuda.is_available():
    dev = 'cuda'
elif torch.backends.mps.is_available():
	dev =  'mps' # pour mon m2 fetiche
else:
    dev = 'cpu'
    
print(f"on evalue sur {dev} !!")

### 2. telechargement des images depuis hugging face

on va chercher les images distillees directement sur le repo hf. comme ca pas besoin de lancer le notebook 1 pour tester :)

In [ ]:
HF_REPO = "jack635/mtt-distillation-buffers"

# on telecharge les images best qu'on a sauve pendant la distillation
# le chemin sur hf c'est logged_files/CIFAR10/run_xxx/images_best.pt
# faut adapter selon ce qui est dispo sur le repo

def download_synset_from_hf(run_name):
    """telecharge les images et labels depuis hf"""
    try :
        img_path = hf_hub_download(
            repo_id=HF_REPO,
            filename=f"logged_files/CIFAR10/{run_name}/images_best.pt",
            repo_type="model"
        )
        lab_path = hf_hub_download(
            repo_id=HF_REPO,
            filename=f"logged_files/CIFAR10/{run_name}/labels_best.pt", 
            repo_type="model"
        )
        print(f"telechargement ok depuis {HF_REPO} :)")
        return img_path, lab_path
    except Exception as e:
        print(f"erreur de telechargement : {e}")
        return None,  None

# a adapter avec le nom de ton run sur hf
# tu peux aller voir sur https://huggingface.co/jack635/mtt-distillation-buffers/tree/main/logged_files
RUN_NAME = "dummy-4frgdagy"  # exemple, a changer

img_file, lab_file = download_synset_from_hf(RUN_NAME)

### 3. chargement des donnees

maintenant on charge les tensors en memoire pour le benchmark.

In [ ]:
class eval_args:
    dataset = 'CIFAR10'
    model = 'ConvNet' # on peut changer pour ResNet18 si on veut rigoler
    epoch_eval_train = 1000 # temps d'entrainement sur le synset
    lr_net = 0.01
    batch_train = 256
    device = dev
    dsa = True
    zca = True # important si on l'a mis a la distillation !!!
    
config = eval_args()

if img_file and lab_file:
    img_syn = torch.load(img_file, map_location=dev)
    lab_syn = torch.load(lab_file, map_location=dev)
    print(f"images chargees : {img_syn.shape}")
    print(f"labels charges : {lab_syn.shape}")
else:
    print("pas de donnees a charger :(")

### 4. entrainement et mesure de l'accuracy

on lance l'entrainement d'un nouveau modele a partir de zero, mais seulement sur nos 10 images (pour ipc=1). 
ensuite on le teste sur les vraies donnees de cifar10 qu'il n'a jamais vues. c'est le moment de verite. :o

In [ ]:
print("chargement du vrai set de test pour la validation...")
_, _, _, _, _, _, _, _, test_loader, _, _, _ = get_dataset(config.dataset, './data', 256, '', args=config)

def run_benchmark(model_name):
    print(f"\nevaluation sur {model_name} en cours...")
    
    # on cree le reseau tout neuf
    net = get_network(model_name, 3, 10, (32,32)).to(dev)
    
    # on utilise la fonction evaluate du repo pour pas se planter dans les calculs
    _, acc_train, acc_test = evaluate_synset(0, net, img_syn, lab_syn, test_loader, config)
    
    print(f"resultat final pour {model_name} :") 
    print(f"   accuracy train (sur le synset) : {acc_train:.2%}")
    print(f"   accuracy test (sur cifar10)  : {acc_test:.2%}")
    
    return acc_test

# on teste sur le modele de base
run_benchmark('ConvNet')

# si t'as le temps essaye avec ca
# run_benchmark('AlexNet') # c'est un peu plus lourd mais ca se fait :p